# Метрики DCG@K и NDCG@K

Входные данные:
- некий запрос **q** (сейчас не важно какой именно)
- коллекция из N текстовых документов **d_i**, i = 1, ..., N (N может быть большим)
- для каждого документа известна его оценка релевантности **qrel(q,d_i)** относительно запроса **q**

Будем обозначать документы строками вида "d1", "d2", ... (на практике обычно употребляются численные docID, но мы тут упростим себе жизнь).

Предположим, что релевантность оценивается по 3-х балльной шкале:
- 1 - не-релевантный документ
- 2 - полезный документ
- 3 - полностью релевантный документ

Импортируем модули которые нам понадобятся впоследствии:

In [1]:
import math
from catboost import utils
from sklearn import metrics

Предположим, что поисковая система в ответ на запрос **q** выдала 5 документов в следующем порядке:<br>
1. d4
2. d2
3. d1
4. d5
5. d3

Будем хранить поисковую выдачу в списке **search_results**:

In [2]:
search_results = ["d4", "d2", "d1", "d5", "d3"]

Предположим, что нам известны оценки релевантности этих документов.<br>
Будем хранить их словаре **qrels**:

In [3]:
qrels = {"d1": 2, "d2": 2, "d3": 2, "d4": 3, "d5": 1}

## Считаем DCG@K и NDCG@K самостоятельно

Вспомним формулу DCG:<br>
<br>
DCG@K = SUM(2^rel_i - 1 / log(1+i)), где<br>
<br>
i - позиция, на которой находится документ в поисковой выдаче (номера позиций начинаются с 1 и заканчиваются K)<br>
rel_i - оценка релевантности документа на i-й позиции


Чтобы рассчитать DCG, нам будет удобно преобразовать список "позиция -> документ" в список "позиция -> оценка".
Напишем для этого функцию-хелпер:

In [4]:
def convert_results(search_results, qrels):
    rels = []
    for docid in search_results:
        rel = qrels[docid]
        rels.append(rel)
    return rels

Преобразуем результаты поиска к удобному виду:

In [5]:
y = convert_results(search_results, qrels)
print(y)

[3, 2, 2, 1, 2]


Теперь попробуем самостоятельно реализовать функцию _dcg(y,k)_ которая вычисляет DCG@K:

In [6]:
def dcg(y, k=10):
    """Computes DCG@k for a single query.

    y is a list of relevance grades sorted by position.
    len(y) could be <= k.
    """
    
    r = 0.
    for i, y_i in enumerate(y):
        p = i + 1 # position starts from 1
        r += (2 ** y_i - 1) / math.log(1 + p, 2)
        if p == k:
            break
    return r

Применим к нашей выдаче:

In [7]:
print(dcg(y,k=5))

11.98402424049139


Обратим внимание, что при k=10 значение DCG@10 будет совпадать со значением DCG@5 т.к. мы останавливаем расчет на 5й позиции (или, что аналогично, забиваем пустые позиции нулями):

In [8]:
print(dcg(y,k=10))

11.98402424049139


Теперь посчитаем для этой же выдачи метрику NDCG@K.

Начнем с того, что тоже вспомним формулу: NDCG@K = DCG@K / MAX_DCG, где MAX_DCG -- максимально возможное DCG, которое достигается при идеальной выдаче, т.е. выдаче, отсортированной в порядке убывания релевантности.

Напишем собственную реализацию _ndcg(y,k)_:

In [9]:
def ndcg(y, k=10):
    """Computes NDCG@k for a single query.

    y is a list of relevance grades sorted by position.
    len(y) could be <= k.
    """
    if len(y) == 0:
        return 0

    # Numerator
    dcg_k = dcg(y, k=k)
    # Denominator
    max_dcg = dcg(sorted(y, reverse=True), k=k)

    # Special case of all zeroes
    if max_dcg == 0:
        return 1.
    
    return dcg_k / max_dcg

Обратите внимание, что у нас появился специальный случай "всех нулей", а именно -- что делать, если в знаменателе у нас получается 0?

Такое возможно, например, в случае, когда:
- мы используем для не-релевантных документов оценку 0
- все документы в поисковой выдаче имеют оценку 0

Обратим внимание, что поисковая выдача, в которой все документы имеет одну и ту же оценку (причем неважно какую), всегда имеет NDCG == 1 -- т.е. как ни пересортировывай в такой выдаче результаты, ее DCG не меняется и всегда достигает своего максимума. Поэтому, для единообразия, постулируем что и выдача из всех нулей должна иметь нулевой NDCG.

Применим _ndcg(y,k)_ к нашему примеру:

In [10]:
print(ndcg(y,k=5))

0.99273940647578


Видим, что наша поисковая выдача близка к идеальной. А для k=2 она и вовсе идеальна:

In [11]:
print(ndcg(y,k=2))

1.0


## Считаем DCG@K и NDCG@K с помощью библиотеки catboost

Сравним наш DCG@K с тем, что выдает **catboost**.

Для этого воспользуемся функцией _catboost.utils.eval_metric_, которая принимает на вход 3 списка:
- список оценок релевантности, которые задают ожидаемый порядок документов для данного запроса
- список предсказаний (скоров) модели ранжирования
- список group id, которые задают принадлежность данного документа к группе, где под группой подразумевается запрос, т.е. все документы группируются по запросу. В нашем примере всего один запрос **q**, поэтому будем считать что у всех документов один и тот же group_id=1


Список оценок релевантности у нас уже есть, это _y_:

In [12]:
print(y)

[3, 2, 2, 1, 2]


Теперь надо сформировать список _y_hat_ предсказаний (скоров) модели, которые задают порядок, в котором документы были возвращены поисковой системой.<br>
В нашем случае в качесте такого скора можно использовать позицию в списке _y_, только "наоборот" -- минимальная позиция соответствует максимальному ранку.

In [13]:
y_hat = [len(y) - i for i in range(len(y))]
print(y_hat)

[5, 4, 3, 2, 1]


Также, подготовим список group_id:

In [14]:
group_id = [1] * len(y)
print(group_id)

[1, 1, 1, 1, 1]


Теперь мы можем воспользоваться функцией _eval_metric_ и посчитать DCG@5:

In [15]:
scores = utils.eval_metric(y, y_hat, 'DCG:top=5;type=Exp', group_id=group_id)
print(scores[0])

11.98402424049139


Также, с помощью катбуста легко посчитать и другие метрики, например NDCG@5:

In [16]:
scores = utils.eval_metric(y, y_hat, 'NDCG:top=5;type=Exp', group_id=group_id)
print(scores[0])

0.99273940647578


Аналогично, посчитаем NDCG@2:

In [17]:
scores = utils.eval_metric(y, y_hat, 'NDCG:top=2;type=Exp', group_id=group_id)
print(scores[0])

1.0


Видим, что результаты совпадают с теми, которые у нас получились вручную!

## Считаем DCG@K и NDCG@K с помощью библиотеки sklearn

Также, для расчета DCG и NDCG мы можем использовать соответствующие функции из библиотеки _scikit-learn_.

Они, в отличие от катбуста, принимают на вход два массива размером n_queries x n_docs, где n_samples -- это число запросов, а n_docs -- число документов в поисковой выдаче:
- первый массив должен содержать оценки релевантности, которые задают ожидаемый порядок документов для данного запроса
- второй массив должен содержать предсказания (скоры) модели ранжирования

Список оценок релевантности у нас уже есть, это y:

In [18]:
print(y)

[3, 2, 2, 1, 2]


И список предсказаний модели тоже есть:

In [19]:
print(y_hat)

[5, 4, 3, 2, 1]


Применим функцию _dcg_score_:

In [20]:
print(metrics.dcg_score([y], [y_hat], k=5))

6.466241679685391


Аналогично, посчитаем NDCG@5:

In [21]:
print(metrics.ndcg_score([y], [y_hat], k=5))

0.9932683086972719


И NDCG@2:

In [22]:
print(metrics.ndcg_score([y], [y_hat], k=2))

1.0


Видим, что результаты немного отличаются от того, что у нас получалось вручную (и от того, что выдавал катбуст!)

Что тут происходит?

Оказывается, _sklearn_ "под капотом" использует немного другую формулу для расчета DCG, а именно вариант без экспоненты в числителе.<br>
В _catboost_ такой вариант DCG/NDCG называется "Base", им можно воспользоваться следующим образом:

In [23]:
scores = utils.eval_metric(y, y_hat, 'DCG:top=5;type=Base', group_id=group_id)
print(scores[0])

6.466241679685391


Теперь все опять совпадает!

Мы же в дальнейшем (и во всех ДЗ!) будем использовать именно экспоненциальный вариант DCG.